## Hướng dẫn cài đặt marlowe-cli

### Các thứ cần chuẩn bị

Tạo một máy ảo Ubuntu với cấu hình như sau:
- CPU architecture 64 bit (x86)
- 4vCPU, 16GB RAM và 50GB SSD nếu chạy testnet

### Cài đặt cardano node

#### Cách 1: Tải file cardano-cli và cardano-node đã được build sẵn (nên dùng)

Download the hydra binaries of cardano-node and cardano-cli from the [release notes](https://github.com/intersectmbo/cardano-node/releases)

In [ ]:
wget https://github.com/IntersectMBO/cardano-node/releases/download/10.1.3/cardano-node-10.1.3-linux.tar.gz

Unzip the tar file

In [ ]:
tar -xvzf cardano-node-10.1.3-linux.tar.gz

Thêm /home/ubuntu/bin/ vào $PATH bằng cách thêm dòng sau vào cuối file ~/.profile

In [ ]:
PATH="/home/ubuntu/bin:$PATH"

Tắt đi bật lại terminal

Test thử xem đã dùng được `cardano-cli` và `cardano-node` chưa

In [ ]:
cardano-cli --version
cardano-node --version

#### Cách 2: Build lại từ source code

Tài liệu hướng dẫn: [Installing the node from source](https://developers.cardano.org/docs/get-started/cardano-node/installing-cardano-node/)

##### Cài đặt ghc và cabal

Chạy 2 lệnh sau để download các packages và tools cần thiết để build được `cardano-node` từ source code

In [ ]:
sudo apt-get update -y
sudo apt-get install automake build-essential pkg-config libffi-dev libgmp-dev libssl-dev libtinfo-dev libsystemd-dev zlib1g-dev make g++ tmux git jq wget libncursesw5 libtool autoconf liblmdb-dev -y

Chạy 2 lệnh sau để cài đặt `GHCup` để có thể install các Haskell tools ([tài liệu hướng dẫn cài đặt GHCup](https://www.haskell.org/ghcup/install/))

In [ ]:
sudo apt-get install curl libffi-dev libffi8 libgmp-dev libgmp10 libncurses-dev libncurses5 libtinfo5

curl --proto '=https' --tlsv1.2 -sSf https://get-ghcup.haskell.org | sh

Tắt đi bật lại terminal

Chạy tiếp các lệnh sau

In [ ]:
ghcup install ghc 8.10.7
ghcup install cabal 3.8.1.0
ghcup set ghc 8.10.7
ghcup set cabal 3.8.1.0

Chọn version của cardano node mà bạn muốn cài đặt ([xem danh sách các tags ở đây](https://github.com/IntersectMBO/cardano-node/tags)). Sau đó chạy 2 lệnh sau (lưu ý là thay 10.1.3 thành cái version bạn muốn cài đặt)

In [ ]:
CARDANO_NODE_VERSION='10.1.3'
IOHKNIX_VERSION=$(curl https://raw.githubusercontent.com/IntersectMBO/cardano-node/$CARDANO_NODE_VERSION/flake.lock | jq -r '.nodes.iohkNix.locked.rev')
echo "iohk-nix version: $IOHKNIX_VERSION"

##### Cài đặt `sodium`

Tạo 1 folder riêng để build sodium

In [ ]:
mkdir -p ~/src
cd ~/src

Tìm version của sodium phù hợp với version của cardano-node

In [ ]:
SODIUM_VERSION=$(curl https://raw.githubusercontent.com/input-output-hk/iohk-nix/$IOHKNIX_VERSION/flake.lock | jq -r '.nodes.sodium.original.rev')
echo "Using sodium version: $SODIUM_VERSION"

Tải và cài đặt sodium

In [ ]:
: ${SODIUM_VERSION:='dbb48cc'}
git clone https://github.com/intersectmbo/libsodium
cd libsodium
git checkout $SODIUM_VERSION
./autogen.sh
./configure
make
make check
sudo make install

Thêm 2 dòng sau vào cuối file ~/.bashrc

In [ ]:
export LD_LIBRARY_PATH="/usr/local/lib:$LD_LIBRARY_PATH"
export PKG_CONFIG_PATH="/usr/local/lib/pkgconfig:$PKG_CONFIG_PATH"

Tắt đi và bật lại terminal

##### Cài đặt `secp256k1`

Tìm version của `secp256k1` phù hợp với version của `cardano-node`

In [ ]:
CARDANO_NODE_VERSION='10.1.3'
IOHKNIX_VERSION=$(curl https://raw.githubusercontent.com/IntersectMBO/cardano-node/$CARDANO_NODE_VERSION/flake.lock | jq -r '.nodes.iohkNix.locked.rev')
echo "iohk-nix version: $IOHKNIX_VERSION"

In [ ]:
SECP256K1_VERSION=$(curl https://raw.githubusercontent.com/input-output-hk/iohk-nix/$IOHKNIX_VERSION/flake.lock | jq -r '.nodes.secp256k1.original.ref')
echo "Using secp256k1 version: ${SECP256K1_VERSION}"

Tải và cài đặt `secp256k1`

In [ ]:
: ${SECP256K1_VERSION:='v0.3.2'}
sudo apt install autoconf
git clone --depth 1 --branch ${SECP256K1_VERSION} https://github.com/bitcoin-core/secp256k1
cd secp256k1
./autogen.sh
./configure --enable-module-schnorrsig --enable-experimental
make
make check
sudo make install

##### Cài đặt blst

Tìm version phù hợp với version của cardano-node

In [ ]:
BLST_VERSION=$(curl https://raw.githubusercontent.com/input-output-hk/iohk-nix/master/flake.lock | jq -r '.nodes.blst.original.ref')
echo "Using blst version: ${BLST_VERSION}"

In [ ]:
cd ~

Tải và cài đặt `blst`

In [ ]:
: ${BLST_VERSION:='v0.3.11'}
git clone --depth 1 --branch ${BLST_VERSION} https://github.com/supranational/blst
cd blst
./build.sh
cat > libblst.pc << EOF
prefix=/usr/local
exec_prefix=\${prefix}
libdir=\${exec_prefix}/lib
includedir=\${prefix}/include

Name: libblst
Description: Multilingual BLS12-381 signature library
URL: https://github.com/supranational/blst
Version: ${BLST_VERSION#v}
Cflags: -I\${includedir}
Libs: -L\${libdir} -lblst
EOF
sudo cp libblst.pc /usr/local/lib/pkgconfig/
sudo cp bindings/blst_aux.h bindings/blst.h bindings/blst.hpp  /usr/local/include/
sudo cp libblst.a /usr/local/lib
sudo chmod u=rw,go=r /usr/local/{lib/{libblst.a,pkgconfig/libblst.pc},include/{blst.{h,hpp},blst_aux.h}}

##### Tải source code của cardano node và build

Tạo thư mục src để build

In [ ]:
mkdir -p ~/src
cd ~/src

Tải source code của cardano node

In [ ]:
git clone https://github.com/intersectmbo/cardano-node.git
cd cardano-node

Check out the latest version of cardano-node (choose the tag with the highest version number: TAGGED-VERSION):

In [ ]:
git fetch --all --recurse-submodules --tags
git tag | sort -V
git checkout tags/<TAGGED VERSION>

Configuring the build options

In [ ]:
echo "with-compiler: ghc-8.10.7" >> cabal.project.local

Building and installing the node (bước này mất khá nhiều thời gian, tầm 4 tiếng)

In [ ]:
cabal update
cabal build all
cabal build cardano-cli

Install the newly built node and CLI commands to the ~/.local/bin directory

In [ ]:
mkdir -p ~/.local/bin
cp -p "$(./scripts/bin-path.sh cardano-node)" ~/.local/bin/
cp -p "$(./scripts/bin-path.sh cardano-cli)" ~/.local/bin/

Thêm ~/.local/bin/ vào $PATH bằng cách thêm dòng sau vào cuối file ~/.profile

In [ ]:
PATH="~/.local/bin:$PATH"

Tắt terminal đi và bật lại, sau đó kiểm tra xem đã dùng được cardano-cli hay chưa bằng cách chạy câu lệnh sau

In [ ]:
cardano-cli --version

### Chạy cardano node

Tạo folder $HOME/cardano/testnet

In [ ]:
mkdir -p $HOME/cardano/testnet
cd $HOME/cardano/testnet

Tải các file config của mạng preprod bằng cách chạy câu lệnh sau

In [ ]:
curl -O -J https://book.play.dev.cardano.org/environments/preprod/config.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/db-sync-config.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/submit-api-config.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/topology.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/byron-genesis.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/shelley-genesis.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/alonzo-genesis.json
curl -O -J https://book.play.dev.cardano.org/environments/preprod/conway-genesis.json

Tạo folder db trong folder $HOME/cardano/testnet/

In [ ]:
mkdir $HOME/cardano/testnet/db

Cấu trúc thư mục sẽ như sau:
```
$HOME/cardano/testnet/
├── alonzo-genesis.json
├── byron-genesis.json
├── config.json
├── conway-genesis.json
├── db
├── db-sync-config.json
├── shelley-genesis.json
├── submit-api-config.json
└── topology.json
```

Chạy node cardano bằng câu lệnh sau

In [ ]:
cardano-node run                                           \
    --config $HOME/cardano/testnet/config.json             \
    --database-path $HOME/cardano/testnet/db/              \
    --socket-path $HOME/cardano/testnet/db/node.socket     \
    --host-addr 0.0.0.0                                    \
    --port 1337                                            \
    --topology $HOME/cardano/testnet/topology.json

Mở một terminal khác, lưu đường dẫn đến node.socket vào biến CARDANO_NODE_SOCKET_PATH bằng cách thêm 2 dòng sau vao file ~/.profile

In [ ]:
export CARDANO_NODE_SOCKET_PATH="/home/ubuntu/cardano/testnet/db/node.socket"
export CARDANO_TESTNET_MAGIC=1 # preprod

Tắt đi và bật lại terminal

Kiểm tra xem node đã bắt đầu đồng bộ dữ liệu chưa bằng câu lệnh sau

In [ ]:
cardano-cli query tip --testnet-magic 1

### Build marlowe-cli từ source code

Cài đặt curl và rsync

In [ ]:
sudo sh -c 'apt update && apt install curl rsync'

Cài đặt z3 (a dependency used by marlowe-cli run analyze for analyzing smart contract safety of contract execution)

In [ ]:
sudo sh -c 'apt install -y z3'

Cài đặt nixOS

In [ ]:
sh <(curl -L https://nixos.org/nix/install) --daemon

Change ownership of the /nix directory to your user ubuntu

In [ ]:
sudo chown -R ubuntu /nix

Run these commands to add the configurations to nix.conf will enhance the build process for NixOS by utilizing pre-compiled, trusted binaries

Documentation: [Setting Up IOG Binary Cache](https://github.com/input-output-hk/iogx/blob/main/doc/nix-setup-guide.md#setting-up-iog-binary-cache)

In [ ]:
sudo sh -c "echo 'substituters = https://cache.nixos.org https://cache.iog.io' >> /etc/nix/nix.conf"
sudo sh -c "echo 'trusted-public-keys = hydra.iohk.io:f/Ea+s+dFdN+3Y/G+FDgSq+a5NEWhJGzdjvKNGv0/EQ= cache.nixos.org-1:6NCHdD59X431o0gWypbMrAURkbJ16ZPMQFGspcDShjY=' >> /etc/nix/nix.conf"
sudo sh -c "echo 'experimental-features = nix-command' >> /etc/nix/nix.conf"  
sudo sh -c "echo 'extra-experimental-features = flakes' >> /etc/nix/nix.conf"  
sudo sh -c 'echo "trusted-users = $0" >> /etc/nix/nix.conf' `whoami`

Restart Nix

In [ ]:
sudo systemctl restart nix-daemon.service

Tắt đi và bật lại terminal

Clone the marlowe-cardano repository

In [ ]:
git clone https://github.com/input-output-hk/marlowe-cardano
cd marlowe-cardano
git checkout 99f432d8ef9dbd1b52b7fa089254de15913b490f

Enter marlowe-cli folder

In [ ]:
cd marlowe-cli

Enter nix virtual environment (~1 hour) be patient!

In [ ]:
nix develop

Download the latest list of Haskell packages (be patient)

In [ ]:
cabal update

Build the marlowe-cli executable

In [ ]:
marlowe-cli

Check the executable file location

In [ ]:
cabal list-bin marlowe-cli

Get out of nix virtual development environment

In [ ]:
exit

Copy the rebuilt version of marlowe-cli to the folder you want

In [ ]:
sudo cp /home/ubuntu/marlowe-cardano/dist-newstyle/build/x86_64-linux/ghc-9.2.8/marlowe-cli-0.2.0.0/x/marlowe-cli/build/marlowe-cli/marlowe-cli ~/.local/bin/marlowe-cli

Test xem marlowe-cli đã đọc được dữ liệu từ node hay chưa. 
_Lưu ý là cần bật cardano node trước khi chạy câu lệnh này_

In [ ]:
marlowe-cli --conway-era util slotting